<a href="https://colab.research.google.com/github/vmahawar/deep-learning-projects/blob/main/LSTM-long-short-term-memory-of-reliance-stock/LSTM_long_short_term_memory_of_reliance_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Load dataset
# Replace 'reliance_stock.csv' with your dataset file
data = pd.read_csv('https://github.com/vmahawar/data-science-datasets-collection/raw/main/reliance_stock.csv')

# Extract closing price and normalize data
closing_prices = data['Close'].values
scaler = MinMaxScaler(feature_range=(0, 1))
closing_prices_scaled = scaler.fit_transform(closing_prices.reshape(-1, 1))

# Function to prepare data with time window
def create_dataset(dataset, time_steps=30):
    X, y = [], []
    for i in range(len(dataset) - time_steps):
        X.append(dataset[i:i + time_steps, 0])
        y.append(dataset[i + time_steps, 0])
    return np.array(X), np.array(y)

# Prepare data for both 30-day and 120-day time windows
time_window = 30  # Change to 120 for the second case
X, y = create_dataset(closing_prices_scaled, time_window)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape data for LSTM (samples, time_steps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build LSTM model
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    LSTM(60, return_sequences=True),
    LSTM(20),
    Dense(1, activation='relu')
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0005), loss='mse')

# Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=1
)

# Evaluate the model
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss (MSE): {test_loss}")

# Predict and inverse scale predictions
predicted_prices = model.predict(X_test)
predicted_prices = scaler.inverse_transform(predicted_prices.reshape(-1, 1))